In [ ]:
from local.torch_basics import *
from local.test import *
from local.core import *
from local.layers import *
from local.data.all import *
from local.optimizer import *
from local.learner import *
from local.metrics import *
from local.text.core import *
from local.text.data import *
from local.text.models.core import *
from local.text.models.awdlstm import *
from local.callback.rnn import *
from local.callback.all import *

In [ ]:
# all_slow

# Integration test on Wikitext-2

> Training a Language Model on WT2

## Data

In [ ]:
path = untar_data(URLs.WIKITEXT_TINY)

The dataset comes with all the wrticles concatenated. We split them to be able to shuffle at the beginning of each epoch.

In [ ]:
def istitle(line):
    return len(re.findall(r'^ = [^=]* = $', line)) != 0

def read_file(filename):
    articles = L()
    with open(filename, encoding='utf8') as f:
        lines = f.readlines()
    current_article = ''
    for i,line in enumerate(lines):
        current_article += line.replace('<unk>', UNK)
        if i < len(lines)-2 and lines[i+1] == ' \n' and istitle(lines[i+2]):
            articles.append(current_article.split(' '))
            current_article = ''
    articles.append(current_article.split(' '))
    return articles

Then we put our list of tokenized texts together in an `LM_Dataset`. It will return tuples of sequences of `seq_len`, with the second sequence between the first one shifted by one on the right.

In [ ]:
trn_txt = read_file(path/'train.txt')
val_txt = read_file(path/'valid.txt')
tst_txt = read_file(path/'test.txt')

In [ ]:
all_texts = np.concatenate([val_txt, trn_txt, tst_txt])
df = pd.DataFrame({'texts':all_texts})
df.head()

,texts
0,"[, \n, =, Homarus, gammarus, =, \n, \n, Homarus, gammarus, ,, known, as, the, European, lobster, or, common, lobster, ,, is, a, species, of, xxunk, lobster, from, the, eastern, Atlantic, Ocean, ,, Mediterranean, Sea, and, parts, of, the, Black, Sea, ., It, is, closely, related, to, the, American, lobster, ,, H., americanus, ., It, may, grow, to, a, length, of, 60, cm, (, 24, in, ), and, a, mass, of, 6, kilograms, (, 13, lb, ), ,, and, bears, a, conspicuous, pair, of, claws, ., In, life, ,, the, lobsters, are, blue, ,, only, becoming, "", lobster, red, "", on, ...]"
1,"[, \n, =, Frank, xxunk, =, \n, \n, Air, Vice, Marshal, Frank, xxunk, ,, CB, ,, CBE, (, 15, July, 1914, –, 23, December, 1976, ), was, a, senior, commander, in, the, Royal, Australian, Air, Force, (, RAAF, ), ., Born, and, educated, in, Tasmania, ,, he, joined, the, RAAF, as, an, air, cadet, in, January, 1934, ., He, specialised, in, flying, instruction, and, navigation, before, the, outbreak, of, World, War, II, ., In, April, 1941, ,, he, became, commanding, officer, of, No., 2, Squadron, ,, which, operated, Lockheed, xxunk, ., The, squadron, was, deployed, to, Dutch, Timor, in, December, ..."
2,"[, \n, =, M, @-@, 82, (, Michigan, highway, ), =, \n, \n, M, @-@, 82, is, a, state, trunkline, in, the, Lower, Peninsula, in, the, US, state, of, Michigan, that, travels, between, xxunk, and, Howard, City, ., The, section, between, xxunk, and, Howard, City, travels, through, xxunk, and, along, the, southern, edge, of, xxunk, National, Forest, ., The, current, version, of, M, @-@, 82, is, actually, the, second, in, the, state, ;, the, first, usage, appeared, in, the, Upper, Peninsula, by, 1919, ., The, Lower, Peninsula, routing, has, been, in, use, since, the, 1920s, ., Various, extensions,..."
3,"[, \n, =, xxunk, xxunk, =, \n, \n, xxunk, xxunk, (, xxunk, xxunk, ,, xxunk, xxunk, ), is, a, fictional, character, in, the, xxunk, manga, and, anime, series, created, by, xxunk, xxunk, ., In, the, anime, and, manga, ,, xxunk, is, a, ninja, affiliated, with, the, village, of, xxunk, ., He, is, a, member, of, Team, 10, ,, a, group, of, ninja, consisting, of, himself, ,, xxunk, xxunk, ,, xxunk, xxunk, ,, and, team, leader, xxunk, xxunk, ., xxunk, is, portrayed, as, a, lazy, character, ,, unwilling, to, apply, his, prodigious, intelligence, ;, xxunk, has, noted, that, he, likes, xxunk, ...]"
4,"[, \n, =, Meridian, ,, Mississippi, =, \n, \n, Meridian, is, the, sixth, largest, city, in, the, state, of, Mississippi, ,, in, the, United, States, ., It, is, the, county, seat, of, Lauderdale, County, and, the, principal, city, of, the, Meridian, ,, Mississippi, xxunk, Statistical, Area, ., Along, major, highways, ,, the, city, is, 93, mi, (, 150, km, ), east, of, Jackson, ,, Mississippi, ;, 154, mi, (, xxunk, km, ), west, of, Birmingham, ,, Alabama, ;, 202, mi, (, 325, km, ), northeast, of, New, Orleans, ,, Louisiana, ;, and, 231, mi, (, 372, km, ), southeast, of, ...]"


In [ ]:
#df_tok,count = tokenize_df(df, ['texts'])

In [ ]:
count = Counter([p for t in df["texts"].values for p in t])
vocab = make_vocab(count)
tfm = Numericalize(make_vocab(count))

In [ ]:
splits = [list(range(len(val_txt), len(df))), list(range(len(val_txt)))]
dsrc = DataSource(df, [[attrgetter("texts"), tfm]], splits=splits, dl_type=LMDataLoader)

In [ ]:
bs,sl = 104,72
dbch = dsrc.databunch(bs=bs, seq_len=sl, after_batch=Cuda)

In [ ]:
dbch.show_batch()

,text
0,"\n = Valkyria Chronicles III = \n \n Senjō no Valkyria 3 : xxunk Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the"
1,""" . He believed that the monument consisted of both a "" chamber "" and an "" oval "" of stones , suggesting that they were "" two distinct xxunk "" . In 1880 , the archaeologist Flinders Petrie included the existence of the stones at "" xxunk "" in his list of xxunk earthworks ; although noting that a previous commentator had described the stones as being in the shape of"
2,"compete with Amanda , while Amanda comments that Tessa is "" ... quite refreshing in a xxunk sort of way . "" xxunk played Amanda 's interaction with Tessa xxunk and a little xxunk . Tessa is usually jealous of MacLeod 's past lovers , but acknowledges in "" Saving Grace "" that "" ... it would take me several lifetimes to find out everything in Duncan 's past . I know"
3,"Communities for freshmen and students with similar interests and concentrations . Participants meet people with similar interests , live on the same floor and take courses with others in their community , while receiving additional guidance related to those interests . The university 's Learning Community programs are divided into two categories , Freshman Learning Communities and Living Learning Communities . The freshman program offers 16 different concentrations , including business ,"
4,"for one week on 17 May 1986 . In the United States , West End Girls debuted on the Billboard Hot 100 at number 71 , reaching the number one position on 10 May 1986 , and remained on the chart for 20 weeks . The song also peaked at number one on Billboard 's Hot Dance Music / Club Play chart for two weeks . \n \n = = Formats and"
5,"At about 3 pm , a single U.S. helicopter arrived on the battlefield to evacuate the casualties , but they only picked up the bodies of the four American crewmen , while South Vietnamese casualties were forced to wait for another helicopter to arrive . At 4 pm , Major Nguyễn Văn xxunk ordered the 4th Marine Battalion to carry their casualties back to the village , instead of continuing to wait"
6,"\n Of the three ships in its class , only Derfflinger was ordered as an addition to the fleet , under the provisional name "" K "" . The other two ships were to intended to replace obsolete vessels ; Lützow was ordered as Ersatz xxunk Augusta for the elderly protected cruiser xxunk Augusta and the contract for Hindenburg was issued under the provisional name Ersatz xxunk , to replace the protected"
7,"Ouw Peh Tjoa ( Chinese : xxunk ; xxunk for Black and White xxunk ) , also known by the Malay @-@ language title xxunk Siloeman Oeler Poeti en Item ( meaning Two xxunk , One White and One Black ) , is a 1934 film from the Dutch East Indies ( now Indonesia ) . It was directed and produced by The Teng Chun . Adapted from Legend of the White"
8,"writers certainly captured the spirit that fuels each of our ( playground ) builds – something that inspires volunteers to continue their great work and strive even harder to give back to their communities . "" \n Within a week of the episode 's original broadcast , two deleted scenes from "" Kaboom "" were made available on the official Parks and Recreation website . The first two @-@ minute clip included"
9,"United Reserves \n The Central League winners : 1947 – 48 \n Chelsea \n Football League First Division winners : 1954 – 55 \n Crystal Palace \n Football League Fourth Division runners @-@ up : 1960 – 61 \n \n \n = Hydnellum peckii = \n \n Hydnellum peckii is an inedible ( though not toxic ) fungus , and a member of the genus Hydnellum of the family Bankeraceae ."


## Model

In [ ]:
config = awd_lstm_lm_config.copy()
config.update({'input_p': 0.6, 'output_p': 0.4, 'weight_p': 0.5, 'embed_p': 0.1, 'hidden_p': 0.2})
model = get_language_model(AWD_LSTM, len(vocab), config=config)

In [ ]:
opt_func = partial(Adam, wd=0.1, eps=1e-7)
cb_funcs = [partial(MixedPrecision, clip=0.1), partial(RNNTrainer, alpha=2, beta=1)]

In [ ]:
learn = Learner(dbch, model, loss_func=CrossEntropyLossFlat(), opt_func=opt_func, cb_funcs=cb_funcs, metrics=[accuracy, Perplexity()])

In [ ]:
learn.fit_one_cycle(1, 5e-3, moms=(0.8,0.7,0.8), div=10)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,7.002256,6.007985,0.170773,406.662903,00:27


Full training

In [ ]:
#learn.fit_one_cycle(90, 5e-3, moms=(0.8,0.7,0.8), div=10)